In [2]:
pip install ucimlrepo

  Using cached ucimlrepo-0.0.6-py3-none-any.whl.metadata (5.3 kB)
Using cached ucimlrepo-0.0.6-py3-none-any.whl (8.0 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
us_census_data_1990 = fetch_ucirepo(id=116)

# data (as pandas dataframes)
X = us_census_data_1990.data.features
y = us_census_data_1990.data.targets


In [1]:
import pandas as pd
import time
pd.set_option('display.max_columns', None)

In [6]:
newX = X
newX['collapsed_income'] = newX.loc[:, 'dIncome1':'dIncome8'].idxmax(axis=1)
newX['income'] = newX['collapsed_income'].str.strip().str[-1].astype(int)
newX = newX.drop(['dIncome1', 'dIncome2', 'dIncome3', 'dIncome4', 'dIncome5', 'dIncome6', 'dIncome7', 'dIncome8', 'collapsed_income'], axis = 1)

In [2]:
import pyarrow as pa
import pyarrow.parquet as pq

In [8]:
start_time = time.time()
table = pa.Table.from_pandas(newX)
pq.write_table(table, 'data2.parquet')
end_time = time.time()

end_time - start_time

4.095423698425293

In [9]:
start_time = time.time()
table = pa.Table.from_pandas(X)
pq.write_table(table, 'data.parquet')
end_time = time.time()

end_time - start_time

4.685370922088623

In [3]:
X = pd.read_parquet('data.parquet')

In [4]:
start_time = time.time()
pd.DataFrame(X).to_json('data.json')
end_time = time.time()

end_time - start_time

26.080443382263184

In [5]:
from pyarrow import orc

In [7]:
start_time = time.time()
pd.DataFrame(newX).to_json('data2.json')
end_time = time.time()

end_time - start_time

27.223031044006348

In [8]:
start_time = time.time()
table = pa.Table.from_pandas(X)
orc.write_table(table, 'data.orc')
end_time = time.time()

end_time - start_time

8.219791173934937

In [9]:
start_time = time.time()
table2 = pa.Table.from_pandas(newX)
orc.write_table(table2, 'data2.orc')
end_time = time.time()

end_time - start_time

7.115490674972534

In [10]:
start_time = time.time()
pd.DataFrame(X).to_csv('data.csv')
end_time = time.time()

end_time - start_time

26.582375526428223

In [11]:
start_time = time.time()
pd.DataFrame(newX).to_csv('data2.csv')
end_time = time.time()

end_time - start_time

22.62074375152588

In [13]:
import os
 
file_size = os.path.getsize('data.json')/(1024*1024)
print("JSON Base file Size is :", file_size, "MB")

JSON Base file Size is : 1921.920265197754 MB


In [15]:
file_size = os.path.getsize('data.json')/(1024*1024)
print("JSON Base file Size is :", file_size, "MB")
file_size = os.path.getsize('data.parquet')/(1024*1024)
print("Paruqet Base file Size is :", file_size, "MB")
file_size = os.path.getsize('data.csv')/(1024*1024)
print("CSV Base file Size is :", file_size, "MB")
file_size = os.path.getsize('data.orc')/(1024*1024)
print("ORC Base file Size is :", file_size, "MB")

JSON Base file Size is : 1921.920265197754 MB
Paruqet Base file Size is : 40.81702423095703 MB
CSV Base file Size is : 368.0190887451172 MB
ORC Base file Size is : 107.80815410614014 MB


In [16]:
file_size = os.path.getsize('data2.json')/(1024*1024)
print("JSON file Size is :", file_size, "MB")
file_size = os.path.getsize('data2.parquet')/(1024*1024)
print("Paruqet file Size is :", file_size, "MB")
file_size = os.path.getsize('data2.csv')/(1024*1024)
print("CSV file Size is :", file_size, "MB")
file_size = os.path.getsize('data2.orc')/(1024*1024)
print("ORC file Size is :", file_size, "MB")

JSON file Size is : 1657.1616973876953 MB
Paruqet file Size is : 37.7671012878418 MB
CSV file Size is : 309.40892219543457 MB
ORC file Size is : 82.28339576721191 MB


In [17]:
import boto3
import sagemaker

# Get region 
session = boto3.session.Session()
region_name = session.region_name

# Get SageMaker session & default S3 bucket
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [19]:
s3 = boto3.resource('s3')


In [20]:
s3.meta.client.upload_file('data2.parquet', 'team12', 'data2.parquet')
s3.meta.client.upload_file('data2.csv', 'team12', 'data2.csv')
s3.meta.client.upload_file('data2.json', 'team12', 'data2.json')
s3.meta.client.upload_file('data2.orc', 'team12', 'data2.orc')

In [21]:
s3.meta.client.upload_file('data.parquet', 'team12', 'data.parquet')
s3.meta.client.upload_file('data.csv', 'team12', 'data.csv')
s3.meta.client.upload_file('data.json', 'team12', 'data.json')
s3.meta.client.upload_file('data.orc', 'team12', 'data.orc')